# Import libraries

Import all necessary libraries

In [1]:
import sys
import os
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt

Now time to ensure that MnistClassifier and function load_and_transform_mnist_data is possible to import without errors.

In [11]:
# The import logic should work if 'mnist_classifier.py' is in the `src/utils/classes/` folder
# and 'load_and_transform_mnist_data.py' is in the 'src/utils/' folder.
try:
    from src.utils.classes.mnist_classifier import MnistClassifier
    print("Successfully imported MnistClassifier.")
except ImportError as e:
    print(f"Import Error: {e}")
    print("Ensure that 'src/utils/classes/mnist_classifier.py' is in the Python path.")

try:
    from src.utils.load_and_transform_mnist_data import load_and_transform_mnist
    print("Successfully imported data loading helper.")
except ImportError as e:
    print(f"Import Error: {e}")
    print("Ensure that 'src/utils/load_and_transform_mnist_data.py' is in the Python path.")

Successfully imported MnistClassifier.
Successfully imported data loading helper.


If error persists, please check the file paths and your PYTHONPATH environment variable.
Then, import theese modules again.

In [8]:
from src.utils.classes.mnist_classifier import MnistClassifier
from src.utils.load_and_transform_mnist_data import load_and_transform_mnist

# Data Loading

In [12]:
X_full, y_full, X_test = load_and_transform_mnist()
print(f"Full Dataset Loaded: X_full shape {X_full.shape}, y_full shape {y_full.shape}")

100%|██████████| 9.91M/9.91M [00:01<00:00, 7.53MB/s]
100%|██████████| 28.9k/28.9k [00:00<00:00, 169kB/s]
100%|██████████| 1.65M/1.65M [00:00<00:00, 2.79MB/s]
100%|██████████| 4.54k/4.54k [00:00<00:00, 884kB/s]

Full Dataset Loaded: X_full shape (60000, 28, 28), y_full shape (60000,)


After successful splitting the dataset, the subset for quick training is created (for instance 10000 samples).

In [13]:
# Prepare a subset for quick training demo (e.g., 5000 samples)
SAMPLE_SIZE = 10000
TEST_SIZE = 0.3

# Create the training/validation split from the subset
X_train_sub, X_val, y_train_sub, y_val = train_test_split(
    X_full[:SAMPLE_SIZE], y_full[:SAMPLE_SIZE], test_size=TEST_SIZE, random_state=42, stratify=y_full[:SAMPLE_SIZE]
)

print("-" * 30)
print(f"Subset for Demo Training: {X_train_sub.shape}")
print(f"Subset for Demo Validation: {X_val.shape}")
print("-" * 30)

------------------------------
Subset for Demo Training: (7000, 28, 28)
Subset for Demo Validation: (3000, 28, 28)
------------------------------
